In [1]:
#imports
using Distributions;
using Random;
using DataFrames;
using CSV;
using Statistics;
using LinearAlgebra;
using Flux;
using Flux: params, 
            Dense, 
            Chain, 
            glorot_normal, 
            normalise, 
            Optimiser,
            train!;

###MC errors and bars for d = 2

function mc_err_bars(T,file_name)
    df_row_titles = DataFrame(x1= "x1",x2 ="x2"
        ,y_vals = "y_vals",mc_vals="mc_vals",sd_err="sd_errs")
    
    CSV.write(file_name, df_row_titles, append = true);

    d = 2
    
    r = Float16(0.05);
    K = Int(100); #strike
    mu = Float16(-0.05); 
    a = 90
    b = 110
    N = 1

    time_grid = LinRange(1/d, 1, d); 
    sigma = 0.1 .+ 0.5 .* time_grid;
    mc_samples = 1000000
    batch_samples = 8000
    batch_test = 400
    
    initial_sampler = Uniform(90,110);

    train_steps = 400000
    learning_rate = 0.001
    learn_rate_decrease = 100000
    
    #standard normalisation function 
    function norm_ab(y)
        mid_point = (a + b) / 2
        y_norm(y) = (y .- mid_point) ./ (b-a)
        return mapslices(y_norm,y;dims =1)
    end
    
    #fixed samples
    X_test_vals = rand(initial_sampler,(d,batch_test))
    
    function x_sde(X::Array,batch) #discretisation of SDE 
        #how_many_samples = length(X)
        for _ in 1:N
            #take appropriate samples from normal 
            eps = rand(Normal(0,1),(d, batch))
            X = X .* (exp.((mu .- 0.5*sigma.^2)*(T/N) .+  sqrt(T/N).*( sigma .* eps)))
        end
        return X
    end;    

    function x_phi(x::Array) #function to use with FK expectation /initial conditions
        phi_(x) = exp(-r*T) * max((maximum(x)-K),0)
        mapslices(phi_, x; dims =1)
    end;
        
    function generate_training_data(X_init,x_sde,x_phi)    
        X_0 = Array(norm_ab(X_init))
        X_sde = x_sde(X_init, batch_samples)
        y_train = x_phi(X_sde)
        return [(X_0,y_train)]
    end

    #define network layers
    input = Dense(d, d + 10, tanh; 
                           bias = true, 
                           init = glorot_normal)

    hidden = Dense(d + 10, d + 10, tanh;
                            bias = true,
                            init = glorot_normal)

    #no activation on the last layer
    output = Dense(d + 10,1,identity)

    batch_norm_layer = BatchNorm(d + 10, identity;
                                            initβ = zeros, 
                                            initγ = ones,
                                            ϵ = 1e-6, 
                                            momentum = 0.9)
    
    #define network architecture
    m = Chain(input,
        #       batch_norm_layer,
                hidden,
        #        batch_norm_layer,
        #        hidden,
        #        batch_norm_layer,
                output)
 
    #loss function = 
    loss(u,v) = mean((m(u) - v).^2)
    
    ps = Flux.params(m)

    opt = Optimiser(ExpDecay(learning_rate,0.01,learn_rate_decrease,1e-8),ADAM()) #optimiser

    #set to train mode
    trainmode!(m)
    
    #generate initial training data
    X_init = rand(initial_sampler,(d,batch_samples))
    data = generate_training_data(X_init,x_sde,x_phi)
    
    for k in 1:train_steps
        
        #generate new training data
        X_init = rand(initial_sampler,(d,batch_samples))
        data = generate_training_data(X_init,x_sde,x_phi)

        #learning step
        train!(loss,ps,data,opt)
    end
            
    testmode!(m)
    
    y_vals = m(norm_ab(X_test_vals))
    
    #generate mc data
    #add sd output here 
    function mc_sampler(X_test_vals)
        x_mc_store = zeros((1,batch_test))
        x_mc_sq = zeros((1,batch_test))
        for _ in 1:mc_samples
            sample_val = Array((x_phi(x_sde(X_test_vals,batch_test))))
            x_mc_store += sample_val
            x_mc_sq += sample_val.^2
        end
        phi_mc = x_mc_store ./ mc_samples;
        phi_mc_sq = x_mc_sq ./ mc_samples;
        phi_mc_mean_sq = phi_mc .^ 2;
        sd_err = vec(sqrt.((phi_mc_sq .- phi_mc_mean_sq)./mc_samples));
        mc_vals = vec(phi_mc);
        
        return mc_vals, sd_err
    end 
    
    mc_ans = mc_sampler(X_test_vals)
    x1 = vec(X_test_vals[1,:])
    x2 = vec(X_test_vals[2,:])
    
    df_row = DataFrame(x1= x1,x2 =x2
        ,y_vals = vec(y_vals),mc_vals=vec(mc_ans[1]),sd_err=vec(mc_ans[2]))
        
    #write errs to file
    CSV.write(file_name, df_row, append = true)
    

end

mc_err_bars(1,"mc_errs_d2_t1_gbm.csv")
mc_err_bars(0.5,"mc_errs_d2_t05_gbm.csv")

"mc_errs_d2_t05_gbm.csv"